In [1]:
import os
import sys
import numpy as np
import sympy as sp
import torch as pt
from torch.nn.functional import normalize
import scipy
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
import logging
import sys
from torch.utils.data import TensorDataset, DataLoader
from torch.func import hessian, vmap
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = pt.device("cuda" if pt.cuda.is_available() else "cpu")

sys.path.append(os.path.abspath('../'))

from src.useful_functions import *
from src.PWDs_module import generate_PWDistances_torch
from src.sqra_functions import*
from src.openmm_functions import*
from src.isokann_modules2 import*
# For reproducibility
np.random.seed(0)


# Read directory paths
read_dirs_paths('dir_paths_.txt', globals())



 
Created variables:
inp_dir = /scratch/htc/fsafarov/structures/8ef5_july_2025/8ef5/
dcd_dir = /scratch/htc/fsafarov/mOR_dcd_files/npat/
out_dir = /scratch/htc/fsafarov/ISOKANN_PINN/output/


In [2]:
psf_file = 'step5_assembly.psf'
crd_file = 'step5_assembly.crd'

system, psf, crd = setup_system(
                    inp_dir,
                    ligand_name = '7V7',
                    nbcutoff = 1.0,
                    psf_file=psf_file,
                    crd_file=crd_file  
                )


dcd_file = 'mOR_simulation_NPAT_CA_aligned.dcd'
dt=2.0
gamma=1.0
T=310.15
platform='CUDA'


forces = get_parameters(
                        system,
                        inp_dir,
                        dcd_file,
                        dcd_dir,
                        dt, #in femtoseconds
                        T,
                        gamma,
                        platform,
                        pdb_file='step5_assembly.pdb',
                        integrator_type='Langevin',
                        psf=psf,
                        get_potential_grad=True
                        
                      )


Number of atoms in the system: 118294


/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.27; phase=180.0; per=2> with <DihedralType: k=0.03; phase=180.0; per=2>
  warnings.warn('Replacing dihedral %r with %r' %
/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.0; phase=0.0; per=3> with <DihedralType: k=0.12; phase=0.0; per=3>
  warnings.warn('Replacing dihedral %r with %r' %


dcdplugin) detected standard 32-bit DCD file of native endianness
dcdplugin) CHARMM format DCD file (also NAMD 2.1 and later)


In [4]:
ca_indices = select(psf=psf,selection="protein and name CA")
forces_ = np.linalg.norm(forces, axis=-1)

forces_ = forces_[:, ca_indices]


In [5]:
coords = get_parameters(
                        system,
                        inp_dir,
                        dcd_file,
                        dcd_dir,
                        dt, #in femtoseconds
                        T,
                        gamma,
                        platform,
                        pdb_file='step5_assembly.pdb',
                        integrator_type='Langevin',
                        psf=psf,
                        get_coords=True
                      )

dcdplugin) detected standard 32-bit DCD file of native endianness
dcdplugin) CHARMM format DCD file (also NAMD 2.1 and later)


In [6]:
positions = np.linalg.norm(coords, axis=-1)
positions = positions[:, ca_indices]

In [7]:
forces_ = pt.Tensor(forces_)
positions = pt.Tensor(positions)

In [8]:
mean_forces = forces_.mean(dim=-1, keepdim=True)
std_forces = forces_.std(dim=-1, keepdim=True)
forces_ = abs(forces_ - mean_forces)/(std_forces + 1e-8)

In [9]:
mean_forces = forces_.mean(dim=-1, keepdim=True)
std_forces = forces_.std(dim=-1, keepdim=True)
forces_ = (forces_ - mean_forces)/(std_forces + 1e-8)

In [11]:
inp_dim = positions.shape[-1]
nodes = [inp_dim, 128, 64, 1] 
base_mlp = MLP(nodes, act_fun='sigmoid')

In [9]:
x = pt.randn(1,25)
inp_dim = x.shape[-1]
nodes = [inp_dim, 128, 64, 1] 
base_mlp = MLP(nodes, act_fun='sigmoid')

In [10]:
def laplacian_operator(model, x, h_val):
    
    h = pt.zeros_like(x) + h_val

    chi_n = model(x)
    chi_plus = model(x + h)
    chi_minus = model(x - h)

    lap_chi = (chi_minus + chi_plus - 2*chi_n)/h**2

    return lap_chi

def generator_action(model, x, forces_fn, D, h):  
   
    grad_chi = nabla_chi(model, x)
    # print(f"Gradient shape: {grad_chi.shape}")
    # print(f"Chi function shape {chi.shape}")

    # None -> model, 0 -> batch dimensions of chi
    lap_chi = vmap(laplacian_operator, in_dims=(None, 0))(model, x, h)  # vmap over batch


    # print(f"Laplacian chi shape: {lap_chi.shape}")
    return chi, (-0.4*forces_fn * grad_chi.squeeze(-1)).sum(dim=1) + D * lap_chi 

In [12]:
delta = laplacian_operator(base_mlp, x, h_val=1e-3)

In [13]:
print(x)

tensor([[ 1.4523,  1.1677, -0.9968, -0.3987, -1.8286,  0.6716,  0.9888,  0.1399,
          1.2472,  0.4880, -0.2469, -0.0560,  0.1225,  1.0847, -0.5656, -1.2593,
          0.4478,  0.2228,  1.4069, -1.2484,  0.5627,  0.3651, -0.0435, -0.5826,
          0.2135]])


In [14]:
delta

tensor([[-0.0298, -0.0298, -0.0298, -0.0298, -0.0298, -0.0298, -0.0298, -0.0298,
         -0.0298, -0.0298, -0.0298, -0.0298, -0.0298, -0.0298, -0.0298, -0.0298,
         -0.0298, -0.0298, -0.0298, -0.0298, -0.0298, -0.0298, -0.0298, -0.0298,
         -0.0298]], grad_fn=<DivBackward0>)